In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
from get_good_and_bad_words import get_good_and_bad_words

In [2]:
from pipeline_utilities_insincere_questions import *
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model.init_sims(replace=True)

[nltk_data] Downloading package punkt to /Users/lduque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = pd.read_csv('train_inisincere.csv', index_col='qid')
X_test = pd.read_csv('test_insincere.csv', index_col='qid')
X = data[['question_text']]
y = data.target
X_train, X_dictionary, y_train, y_dictionary = train_test_split(X, y, test_size=0.2, random_state=42)
data_dictionary =  pd.concat([X_dictionary, y_dictionary], axis=1)

In [4]:
%%time
good_words, bad_words = get_good_and_bad_words(data_dictionary)
bad_words = list(bad_words.sort_values(ascending=False).index)
good_words = list(good_words.sort_values(ascending=False).index)

size_sublists = 300
chunk_bad_words = [bad_words[x:x+size_sublists] for x in xrange(0, len(bad_words), size_sublists)]
chunk_good_words = [good_words[x:x+size_sublists] for x in xrange(0, len(good_words), size_sublists)]
worst_sentence = ' '.join(chunk_bad_words[0])
best_sentence = ' '.join(chunk_good_words[0])

CPU times: user 4min 55s, sys: 3.91 s, total: 4min 59s
Wall time: 5min 1s


In [5]:
print worst_sentence

muslims liberals democrats women trump white americans indians men castrated people shithole christians castrate brainwashed whites obama hindus hillary racist muslim gay jews feminists black hate blacks supporters conservatives atheists bhakts liberal rape fuck republicans girls damn asians realize sex leftists stupid donald pakistanis enslaved clinton rapists palestinians chinese raping raped indian terrorists gays superior stereotyped incest gujaratis mullahs israel idiots modi islam israelis castrating quora homosexuals europeans racists ignorant dumb shit president lynched fucking believe america tamilians holocaust jewish depraved asian slaughtering kannadigas american shabbat zealanders guys crimes males illegals wives violent brahmins assholes pakistani dick homosexuality implying africans racism democrat guns bullshit jew terrorist kill deny islamist hypocritical xenophobic immigrants hypocrites zionism sister islamic true muhajirs islamization minorities homosexual tamils fac

In [6]:
print best_sentence

best get good job difference work someone engineering possible business online use college book company used university life learn exam computer student marks software app career study learning data one course different important jee buy causes engineer find examples school rank prepare way degree start science experience tips ways water books marketing write process design would year major phone website cost affect system music management car android google invest advantages using studying mobile physics students movie visa program iit required value application month product number time main training exams account favorite long energy effective model market salary mean type test apply interview top institute writing download class web admission companies without earn current programming happen investment meaning cbse advice travel colleges field types price improve medical happens card neet space option score series code starting uses youtube bank describe first service laptop role s

In [7]:
X=[]
y=[]
data=[]
good_words=[]
bad_words=[]
X_dictionary =[]
y_dictionary =[]

In [8]:
%%time
m = stringTransformer(colname='question_text', model=model)
encoded_strings_train = m.transform(X_train)
encoded_strings_test = m.transform(X_test)

CPU times: user 12min 47s, sys: 11.7 s, total: 12min 58s
Wall time: 13min 11s


In [9]:
%%time
m = stringComparison(col='question_text', sentence=worst_sentence, suffix='_worst', model=model, norm_model=norm_model)
comparison_with_worst_sentence_train = m.transform(X_train)
comparison_with_worst_sentence_test = m.transform(X_test)

UnboundLocalError: local variable 'embedded_sentence' referenced before assignment

In [ ]:
%%time
m = stringComparison(col='question_text', sentence=best_sentence, suffix='_best',model=model, norm_model=norm_model)
comparison_with_best_sentence_train = m.transform(X_train)
comparison_with_best_sentence_test = m.transform(X_test)

In [ ]:
train_features = pd.concat([X_train.question_text, encoded_strings_train, comparison_with_worst_sentence_train, comparison_with_best_sentence_train], axis=1)
test_features = pd.concat([X_test.question_text, encoded_strings_test, comparison_with_worst_sentence_test, comparison_with_best_sentence_test], axis=1)

In [ ]:
train_features.to_csv("train_features.csv")
test_features.to_csv("test_features.csv")